In [29]:
#!pip install beautifulsoup4 requests

In [30]:

import requests
from bs4 import BeautifulSoup

In [31]:

url = 'https://Your-url'
headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'}

r = requests.get("https://www.materialepyramiden.dk/",headers=headers,timeout=10)
soup = BeautifulSoup(r.content)
print(soup)


<!DOCTYPE html>

<html lang="da_DA" xml:lang="da" xmlns="http://www.w3.org/1999/xhtml">
<head>
<meta content="sustainability, construction materials, pyramid, pyramide, EPD, livscyklus, lifecycle, analysis, architecture, construction, byggeriet, bygninger, tool, værktøj, digital, materialer, bæredygtighed, livscyklusanalyse, LCA, KADK, Cinark, Vandkunsten" name="keywords"/>
<meta content="KADK/CINARK" name="Copyright"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="text/css" http-equiv="Content-Style-Type"/>
<meta content="text/javascript" http-equiv="Content-Script-Type"/>
<meta content="width=device-width, initial-scale=0.86, minimal-ui" id="viewport" name="viewport"/>
<meta content="yes" name="apple-mobile-web-app-capable"/>
<meta content="black" name="apple-mobile-web-app-status-bar-style"/>
<script charset="UTF-8" src="https://ajax.googleapis.com/ajax/libs/jquery/3.5.1/jquery.min

In [32]:

'''
with open("C:\\Users\\frans\\OneDrive\\Dev_OneDrive\\Rhino\\GH_LCA\\pytamiden\\index.html",encoding='utf8') as fp:
    soup = BeautifulSoup(fp, "html")
'''

'\nwith open("C:\\Users\\frans\\OneDrive\\Dev_OneDrive\\Rhino\\GH_LCA\\pytamiden\\index.html",encoding=\'utf8\') as fp:\n    soup = BeautifulSoup(fp, "html")\n'

In [33]:

results = soup.find_all("div", class_="mat-overlay")
links =[]
for i in range(len(results)):
    links.append(results[i].attrs['data-link'])

print(links)
print(len(links))

['https://materialepyramiden.dk/?materialer=pandeplade', 'https://materialepyramiden.dk/?materialer=maling-mat', 'https://materialepyramiden.dk/?materialer=betontagsten', 'https://materialepyramiden.dk/?materialer=gulvparket', 'https://materialepyramiden.dk/?materialer=mdf', 'https://materialepyramiden.dk/?materialer=poroton-tegl', 'https://materialepyramiden.dk/?materialer=grantrae', 'https://materialepyramiden.dk/?materialer=egetrae', 'https://materialepyramiden.dk/?materialer=modificeret-trae', 'https://materialepyramiden.dk/?materialer=tegl-roed-dobbeltbraendt', 'https://materialepyramiden.dk/?materialer=tegl-roed-enkeltbraendt', 'https://materialepyramiden.dk/?materialer=beton-c20-25', 'https://materialepyramiden.dk/?materialer=aluvindue', 'https://materialepyramiden.dk/?materialer=traevindue', 'https://materialepyramiden.dk/?materialer=trae-alu-vindue', 'https://materialepyramiden.dk/?materialer=fibercement-plader-18-trae', 'https://materialepyramiden.dk/?materialer=kalkpuds-inte

In [34]:
# DOWNLOAD ALL MATERIAL FILES

for i in range(len(links)):
    name = links[i].split("=")[-1]
    

    url = links[i]
    print(f"downloading {name} from {url}")
    ua = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
    response  = requests.get(url, headers={"User-Agent": ua})
    if response.status_code == 200:
        with open(f"./materialer/{name}.html","wb") as file:
            file.write(response.content)
            file.close()
    else:
        print("Bad response code :", response.status_code, url)
    

print("Done")

downloading pandeplade from https://materialepyramiden.dk/?materialer=pandeplade
downloading maling-mat from https://materialepyramiden.dk/?materialer=maling-mat
downloading betontagsten from https://materialepyramiden.dk/?materialer=betontagsten
downloading gulvparket from https://materialepyramiden.dk/?materialer=gulvparket
downloading mdf from https://materialepyramiden.dk/?materialer=mdf
downloading poroton-tegl from https://materialepyramiden.dk/?materialer=poroton-tegl
downloading grantrae from https://materialepyramiden.dk/?materialer=grantrae
downloading egetrae from https://materialepyramiden.dk/?materialer=egetrae
downloading modificeret-trae from https://materialepyramiden.dk/?materialer=modificeret-trae
downloading tegl-roed-dobbeltbraendt from https://materialepyramiden.dk/?materialer=tegl-roed-dobbeltbraendt
downloading tegl-roed-enkeltbraendt from https://materialepyramiden.dk/?materialer=tegl-roed-enkeltbraendt
downloading beton-c20-25 from https://materialepyramiden.dk

In [39]:
# import OS module

import os
from tkinter.tix import DirList
import sqlite3
 
# Get the list of all files and directories
path = "./materialer/"
dir_list = os.listdir(path)
sqliteConnection = sqlite3.connect('./dataBase/materialDB_dev_new.db')

cursor = sqliteConnection.cursor()
tableName = "materials"

query = f"DROP TABLE {tableName}"
#cursor.execute(query)

query = f"""CREATE TABLE IF NOT EXISTS {tableName}(
    name	TEXT NOT NULL UNIQUE,
    category TEXT NOT NULL,
    density NUMERIC NOT NULL,
    GWP NUMERIC NOT NULL,
    ODB NUMERIC NOT NULL,
    POCP NUMERIC NOT NULL,
    EP NUMERIC NOT NULL,
    AP NUMERIC NOT NULL,
	PRIMARY KEY("name")
);"""
cursor.execute(query)


cloumnList = "name,category,density,GWP,ODB,POCP,EP,AP"

#for i in range(len(dir_list)):
runLen = len(dir_list)
#runLen = 1
for i in range(runLen):
    filepath = f"{path}{dir_list[i]}"
    print(f"opening {filepath}")
    with open(filepath, "r",encoding='utf8') as file:
        soup = BeautifulSoup(file, "html")

        name = soup.find( class_ = "single-title single-text c1" ).find(class_ = "en").text
        print(f"found name {name}")

        density = ""
        insulation = ""
        cat = ""
        GWP = ""
        ODP = ""
        POCP  = ""
        EP = ""
        AP = ""



        #table = soup.find('table',class_ = "single-table")
        rows = soup.find_all('tr', class_ = 'tr-line')
        print(len(rows))

        
        for row in rows:
            #print(row)
            #print("   ")
            if(row.find(class_ = 'en')):
                if(row.find(class_ = 'en').text == "Density:"):
                    density = row.find(class_ = 't-text t-number t4').text.split(" ")[0]
            
            if(row.find(class_ = 'en')):
                if(row.find(class_ = 'en').text == "Materialgroup:"):
                    cat = row.find(class_ = 't-text t-number t4').text
            
            if(row.find(class_ = 't-text t1')):
                if(row.find(class_ = 't-text t1').text == "Global Warming Potential [GWP]"):
                    if(row.find(class_ = 't-text t3').text.split("/")[-1] == "m3"):
                        GWP = row.find(class_ = 't-text t-number t2').text

            if(row.find(class_ = 't-text t1')):
                if(row.find(class_ = 't-text t1').text == "Ozone Depletion Potential [ODP]"):
                    if(row.find(class_ = 't-text t3').text.split("/")[-1] == "m3"):
                        ODP = row.find(class_ = 't-text t-number t2').text

            if(row.find(class_ = 't-text t1')):
                if(row.find(class_ = 't-text t1').text == "Photochemical Ozone Depletion Potential [POCP]"):
                    if(row.find(class_ = 't-text t3').text.split("/")[-1] == "m3"):
                        POCP = row.find(class_ = 't-text t-number t2').text

            if(row.find(class_ = 't-text t1')):
                if(row.find(class_ = 't-text t1').text == "Eutrophication Potential [EP]"):
                    if(row.find(class_ = 't-text t3').text.split("/")[-1] == "m3"):
                        EP = row.find(class_ = 't-text t-number t2').text
       
            if(row.find(class_ = 't-text t1')):
                if(row.find(class_ = 't-text t1').text == "Acidification Potential [AP]"):
                    if(row.find(class_ = 't-text t3').text.split("/")[-1] == "m3"):
                        AP = row.find(class_ = 't-text t-number t2').text



            #print("   ")


        print(f"name:{name} , category:{cat}, density:{density}, GWP:{GWP}, ODP:{ODP},POCP:{POCP},EP:{EP},AP:{AP}" )
        
        query = f"""INSERT OR REPLACE INTO {tableName} ({cloumnList})
                    VALUES ('{name}' , '{cat}' , {density} , '{GWP}','{ODP}','{POCP}','{EP}','{AP}');"""
        
        print(query)
        cursor.execute(query)


        

        

      
result = cursor.fetchall()
print('SQLite Version is {}'.format(result))



sqliteConnection.commit()
cursor.close()

opening ./materialer/aluminum-plade.html
found name Aluminium sheet
24
name:Aluminium sheet , category:metal, density:2700.00, GWP:2.82E+4, ODP:1.19E-1,POCP:5.41E+3,EP:6.42E+3,AP:8.75E+4
INSERT OR REPLACE INTO materials (name,category,density,GWP,ODB,POCP,EP,AP)
                    VALUES ('Aluminium sheet' , 'metal' , 2700.00 , '2.82E+4','1.19E-1','5.41E+3','6.42E+3','8.75E+4');
opening ./materialer/aluvindue.html
found name Aluminium frame window
24
name:Aluminium frame window , category:komponenter, density:323.74, GWP:1.17E+3, ODP:1.09E+1,POCP:1.68E+1,EP:5.06E+2,AP:5.53E+3
INSERT OR REPLACE INTO materials (name,category,density,GWP,ODB,POCP,EP,AP)
                    VALUES ('Aluminium frame window' , 'komponenter' , 323.74 , '1.17E+3','1.09E+1','1.68E+1','5.06E+2','5.53E+3');
opening ./materialer/beton-c20-25.html
found name Concrete C20/25
24
name:Concrete C20/25 , category:mineralsk, density:2212.00, GWP:2.29E+2, ODP:2.65E+3,POCP:1.07E+1,EP:1.15E+2,AP:3.70E+2
INSERT OR REPLACE I